# Cleaning Student Addresses and SAM CSV, formatting

In [ ]:
#import libraries
import pandas as pd
#open addresses csv
df = pd.read_csv("addresses.csv")
df2 = pd.read_csv("SAM.csv")
df.head()
print(df2.shape)
df2.head()

In [ ]:
#extracting ears of interest, leases starting from 2018 to 2024
years_of_interest = [ "2019", "2020", "2021", "2022", "2023", "2024"]
pattern = '|'.join(years_of_interest)
matching_rows = df[df['year'].str.contains(pattern, case=False, na=False)]
SAM_matching_rows = df2[df2['last_edited_date'].str.contains(pattern, case=False, na=False)]    

# Combine all address cols with a space delimiter
matching_rows['full address'] = matching_rows['6a. street #'] + ' ' + matching_rows['6b. street name']+ ' ' + matching_rows['6c. street suffix']

SAM_matching_rows = SAM_matching_rows.drop(columns = ['shape_wkt', 'POINT_X', 'POINT_Y', 'X_COORD', 'Y_COORD'])
matching_rows = matching_rows.drop(columns=['6a. street #', '6b. street name', '6c. street suffix'])


display(matching_rows)
display(SAM_matching_rows)
print(SAM_matching_rows.shape)
unique_unis = df['university'].unique()
print(unique_unis)

#test_row = df.loc[(df['6a. street #'] == '10')& (df['6b. street name'] == 'Higgins') & (df['6c. street suffix'] == 'ST')	]
#display(test_row)

matching_rows.to_csv('cleaned_student_addresses.csv', index=False)

#one SAM ID FOR ONE LANDLORD
list(SAM_matching_rows)
#num unique landlords
unique_landlords = SAM_matching_rows['SAM_ADDRESS_ID'].nunique()
print(f'Number of unique landlords: {unique_landlords}')

In [ ]:
#extracting ONLY sam ids from student addresses
matching_rows = matching_rows.rename(columns={'full address': 'FULL_ADDRESS'})
df_matched = matching_rows.merge(SAM_matching_rows, on='FULL_ADDRESS', how='inner')

#keep only cols we want
col_keep = ['6d. unit #',
 '6e. zip',
 '9. 5 or more undergrads/unit (y/n)',
 'year',
 'FULL_ADDRESS',
 'SAM_ADDRESS_ID',
 'BUILDING_ID',
 'STREET_NUMBER',
 'UNIT',
 'ZIP_CODE',
 'WARD',
 'PARCEL',
 'created_date',
 'last_edited_date']

df_matched = df_matched[col_keep]
#unique student housing addresse
num_unique_student_addresses = matching_rows['FULL_ADDRESS'].nunique()
print(f'Number of unique student housing addresses: {num_unique_student_addresses}')
list(df_matched.columns.values)
df_matched.head()

#to csv
df_matched.to_csv('cleaned_student_addresses_SAM.csv', index=False)



# Merging Everything

This section merges the following datasets into one coherent one we will use: 
1. cleaned_student_addresses_SAM.csv
2. cleaned_311_calls.csv
3. cleaned_building_and_property_violations.csv
4. cleaned_property_assessment.csv


In [2]:
import pandas as pd
df_matched = pd.read_csv("cleaned_student_addresses_SAM.csv")
threeoneone = pd.read_csv("cleaned_311_calls.csv")
bpviolations = pd.read_csv("cleaned_building_and_property_violations.csv")
propassessments = pd.read_csv("cleaned_property_assessment.csv")

C:\Users\shery\AppData\Local\Temp\ipykernel_28304\2419804850.py:5: DtypeWarning: Columns (2,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  propassessments = pd.read_csv("cleaned_property_assessment.csv")


In [3]:
import re

suffix_map = {
    'street': 'st', 'st': 'st', 'st.': 'st',
    'road': 'rd',   'rd': 'rd', 'rd.': 'rd',
    'avenue': 'ave','ave': 'ave','ave.': 'ave',
    'boulevard': 'blvd', 'blvd': 'blvd',
    'drive': 'dr', 'dr': 'dr',
    'lane': 'ln', 'ln': 'ln',
    'court': 'ct','ct': 'ct',
    'place': 'pl','pl': 'pl',
    'circle': 'cir','cir': 'cir'
}


# Build a regex pattern to match street address ending with a known suffix
suffix_pattern = '|'.join(suffix_map.keys())
street_regex = re.compile(r'(\d+\s+[\w\s]+?\s+(?:' + suffix_pattern + r'))', flags=re.IGNORECASE)



def normalize_addr(addr):
    # lowercase & strip
    
    a = addr.lower().strip()
    
    # take everything BEFORE the first comma
    a = a.split(',')[0]
    
    # remove punctuation (except spaces and digits)
    a = re.sub(r'[^a-z0-9 ]', ' ', a)
    
    if pd.isna(addr):
        return ""
    match = street_regex.search(addr)
    if not match:
        return addr.lower().strip()  # fallback
    street = match.group(1).lower().strip()
    
    # standardize suffix
    parts = street.split()
    last = parts[-1]
    if last in suffix_map:
        parts[-1] = suffix_map[last]
    return " ".join(parts)


In [4]:
propassessments['full_address'] = propassessments['full_address'].apply(normalize_addr)
threeoneone['location'] = threeoneone['location'].apply(normalize_addr)
bpviolations['full_address'] = bpviolations['full_address'].astype(str).apply(normalize_addr)
df_matched['FULL_ADDRESS'] = df_matched['FULL_ADDRESS'].apply(normalize_addr)

#normalize names
propassessments = propassessments.rename(columns={'full_address': 'FULL_ADDRESS'})
bpviolations = bpviolations.rename(columns={'full_address': 'FULL_ADDRESS'})
threeoneone = threeoneone.rename(columns={'location': 'FULL_ADDRESS'})

bpviolations['status_dttm'] = bpviolations['status_dttm'].astype(str).str[:4]
bpviolations = bpviolations.rename(columns={'status_dttm':'year'})

threeoneone['open_dt'] = threeoneone['open_dt'].astype(str).str[:4]
threeoneone = threeoneone.rename(columns={'open_dt':'year'})

df_matched["year"] = df_matched["year"].astype(str).str[:4]

# type checking 
df_matched["year"] = df_matched["year"].astype(str).replace("nan", "")
threeoneone["year"] = threeoneone["year"].astype(str).replace("nan", "")
bpviolations["year"] = bpviolations["year"].astype(str).replace("nan", "")
propassessments["year"] = propassessments["year"].astype(str).replace("nan", "")

df_matched["FULL_ADDRESS"] = df_matched["FULL_ADDRESS"].astype(str).replace("nan", "")
threeoneone["FULL_ADDRESS"] = threeoneone["FULL_ADDRESS"].astype(str).replace("nan", "")
bpviolations["FULL_ADDRESS"] = bpviolations["FULL_ADDRESS"].astype(str).replace("nan", "")

In [5]:
print("threeoneone:")
display(threeoneone.head())

print("bpviolations:")
display(bpviolations.head())

print("propassessments:")
display(propassessments.head())

threeoneone:


,_id,case_enquiry_id,year,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,type,queue,department,FULL_ADDRESS,pwd_district,neighborhood,neighborhood_services_district,ward
0,1.0,101002296888,2018,NaN,2018-01-02 08:05:36,ONTIME,Closed,Case Closed. Closed date : 2018-01-02 13:05:36...,Request for Snow Plowing (Emergency Responder),Public Works Department,Street Cleaning,Request for Snow Plowing (Emergency Responder),PWDx_District 02: Jamaica Plain,PWDx,342 hyde park ave,02,Jamaica Plain,10,Ward 19
1,2.0,101002296890,2018,2018-01-02 03:30:00,2018-01-01 01:44:28,ONTIME,Closed,Case Closed. Closed date : 2018-01-01 06:44:28...,Street Light Knock Downs,Public Works Department,Street Lights,Street Light Knock Downs,PWDx_Street Light Knock Downs,PWDx,intersection of franklin park rd & glen ln ja...,02,Greater Mattapan,13,12
2,3.0,101002296891,2018,NaN,2018-02-05 06:03:35,ONTIME,Closed,Case Closed. Closed date : 2018-02-05 11:03:35...,General Request,Boston Water & Sewer Commission,Administrative & General Requests,BWSC General Request,BWSC_All Cases,BWSC,134 marion st,09,East Boston,1,Ward 1
3,4.0,101002296892,2018,2018-01-03 03:30:00,2018-01-01 02:27:44,ONTIME,Closed,Case Closed. Closed date : 2018-01-01 07:27:44...,Request for Snow Plowing,Public Works Department,Street Cleaning,Request for Snow Plowing,PWDx_District 03: North Dorchester,PWDx,1080 dorchester ave,03,Dorchester,7,Ward 13
4,5.0,101002296894,2018,NaN,2018-02-23 05:01:35,ONTIME,Closed,Case Closed. Closed date : 2018-02-23 10:01:35...,BWSC Pothole,Boston Water & Sewer Commission,Pothole,BWSC Pothole,BWSC_Pothole,BWSC,125 nashua st,1B,Boston,3,03


bpviolations:


,_id,case_no,ap_case_defn_key,year,status,code,value,description,violation_sthigh,ward,FULL_ADDRESS
0,1,V91983,1013,,Closed,121.2,NaN,Unsafe and Dangerous,NaN,01,302 sumner st
1,2,V876069,1013,2025,Open,102.8,NaN,Maintenance,NaN,15,46 adams st
2,3,V876068,1013,2025,Open,102.8,NaN,Maintenance,NaN,18,70 birchcroft rd
3,4,V876066,1013,2025,Open,102.8,NaN,Maintenance,NaN,07,633 eighth st
4,5,V876048,1013,2025,Open,116,NaN,Unsafe Structures,NaN,18,74 birchcroft rd


propassessments:


,year,PID,CM_ID,GIS_ID,UNIT_NUM,OWNER,INT_COND,EXT_COND,OVERALL_COND,BDRM_COND,HEAT_TYPE,AC_TYPE,BED_RMS,FULL_ADDRESS
0,2024,100001000,NaN,100001000.0,NaN,PASCUCCI CARLO,A - Average,F - Fair,A - Average,NaN,W - Ht Water/Steam,N - None,6.0,104 putnam st
1,2024,100002000,NaN,100002000.0,NaN,SEMBRANO RODERICK,A - Average,A - Average,A - Average,NaN,F - Forced Hot Air,C - Central AC,3.0,197 lexington st
2,2024,100003000,NaN,100003000.0,NaN,GUERRA CHEVARRIA ANA S,A - Average,G - Good,A - Average,NaN,S - Space Heat,N - None,5.0,199 lexington st
3,2024,100004000,NaN,100004000.0,NaN,JB REALTY TRUST,A - Average,A - Average,A - Average,NaN,W - Ht Water/Steam,N - None,5.0,201 lexington st
4,2024,100005000,NaN,100005000.0,NaN,MARKS TRAVIS JOSEPH,A - Average,F - Fair,A - Average,NaN,W - Ht Water/Steam,N - None,6.0,203 lexington st


In [6]:
print(df_matched.shape)
print(propassessments.shape)
print(threeoneone.shape)

print(df_matched['FULL_ADDRESS'].duplicated().sum())
print(propassessments['FULL_ADDRESS'].duplicated().sum())
print(threeoneone['FULL_ADDRESS'].duplicated().sum())

(62807, 14)
(1241119, 14)
(1872649, 19)
54441
1018943
1751856


In [7]:
df_merged_test = df_matched.copy() #copy just in case... 
df_address_311 = df_merged_test.merge(threeoneone, on=['FULL_ADDRESS', "year"], how='left')
df_add_propassessments = df_address_311.merge(propassessments, on=['FULL_ADDRESS', "year"], how='left')
df_merge_total = df_add_propassessments.merge(bpviolations, on=['FULL_ADDRESS', "year"], how='left')

print(df_merge_total.shape)
list(df_merge_total.columns.values)

(3092491, 52)


['6d. unit #',
 '6e. zip',
 '9. 5 or more undergrads/unit (y/n)',
 'year',
 'FULL_ADDRESS',
 'SAM_ADDRESS_ID',
 'BUILDING_ID',
 'STREET_NUMBER',
 'UNIT',
 'ZIP_CODE',
 'WARD',
 'PARCEL',
 'created_date',
 'last_edited_date',
 '_id_x',
 'case_enquiry_id',
 'sla_target_dt',
 'closed_dt',
 'on_time',
 'case_status',
 'closure_reason',
 'case_title',
 'subject',
 'reason',
 'type',
 'queue',
 'department',
 'pwd_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward_x',
 'PID',
 'CM_ID',
 'GIS_ID',
 'UNIT_NUM',
 'OWNER',
 'INT_COND',
 'EXT_COND',
 'OVERALL_COND',
 'BDRM_COND',
 'HEAT_TYPE',
 'AC_TYPE',
 'BED_RMS',
 '_id_y',
 'case_no',
 'ap_case_defn_key',
 'status',
 'code',
 'value',
 'description',
 'violation_sthigh',
 'ward_y']

In [8]:
final_drop = [
    '6d. unit #', 
    '6e. zip',
    'STREET_NUMBER', 
    'UNIT',
    'ZIP_CODE',
    '_id_x',
    'queue',
    'pwd_district',
    'neighborhood',
    'neighborhood_services_district',
    'ward_x',
    'UNIT_NUM',
    "violation_sthigh",
    'ward_y',
    '_id_y',
    'case_no',
    'ap_case_defn_key',
    'status',
    'code',
    'value',
    'description']
df_merged = df_merge_total.drop(final_drop, axis=1)

df_merged = df_merged.rename(columns={'9. 5 or more undergrads/unit (y/n)':'over_5'}) 
df_merged = df_merged.rename(columns={'FULL_ADDRESS':'full_address'}) 
df_merged = df_merged.rename(columns={'SAM_ADDRESS_ID':'sam_id'}) 
df_merged = df_merged.rename(columns={'BUILDING_ID':'building_id'}) 
df_merged = df_merged.rename(columns={'WARD':'ward_id'}) 
df_merged = df_merged.rename(columns={'PARCEL':'parcel_num'}) 
df_merged = df_merged.rename(columns={'created_date':'case_created_date'}) 
df_merged = df_merged.rename(columns={'last_edited_date':'last_case_update'}) 
df_merged = df_merged.rename(columns={'sla_target_dt':'targeted_deadline'}) 
df_merged = df_merged.rename(columns={'closed_dt':'close_date'}) 
df_merged = df_merged.rename(columns={'on_time':'case_met_deadline'}) 
df_merged = df_merged.rename(columns={'subject':'case_subject'}) 
df_merged = df_merged.rename(columns={'reason':'case_reason'}) 
df_merged = df_merged.rename(columns={'type':'case_type'}) 
df_merged = df_merged.rename(columns={'department':'case_department'}) 
df_merged = df_merged.rename(columns={'PID':'p_id'}) 
df_merged = df_merged.rename(columns={'CM_ID':'cm_id'}) 
df_merged = df_merged.rename(columns={'GIS_ID':'gis_id'}) 
df_merged = df_merged.rename(columns={'OWNER':'landlord_name'}) 
df_merged = df_merged.rename(columns={'INT_COND':'int_cond'}) 
df_merged = df_merged.rename(columns={'EXT_COND':'ext_cond'}) 
df_merged = df_merged.rename(columns={'OVERALL_COND':'overall_cond'}) 
df_merged = df_merged.rename(columns={'BDRM_COND':'bdrm_cond'}) 
df_merged = df_merged.rename(columns={'HEAT_TYPE':'heat_type'}) 
df_merged = df_merged.rename(columns={'AC_TYPE':'ac_type'}) 
df_merged = df_merged.rename(columns={'BED_RMS':'num_bed_rms'}) 

list(df_merged.columns.values)

print(df_merged.isna().sum())

over_5               2823596
year                       0
full_address               0
sam_id                     0
building_id                0
ward_id                    0
parcel_num                 0
case_created_date          6
last_case_update           0
case_enquiry_id        17034
targeted_deadline     433129
close_date            218100
case_met_deadline      17041
case_status            17034
closure_reason         17034
case_title             17034
case_subject           17034
case_reason            17034
case_type              17034
case_department        17034
p_id                  272831
cm_id                 353065
gis_id                273139
landlord_name         272929
int_cond              869430
ext_cond              812265
overall_cond          308394
bdrm_cond             907120
heat_type             869427
ac_type               869426
num_bed_rms           830838
dtype: int64


In [9]:
df_merged.to_csv("raw_merged.csv")